### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []
for page_num in range(1, 400):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/tai-chinh-ngan-hang-dau-tu-cong-thuong/t1/14/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
    

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_taichinh_nganhang = pd.DataFrame(articles_data)
df_taichinh_nganhang.head(5)

,Name,Time,Question,Situation,Answer
0,Trung tâm Văn hóa tỉnh,07:05 12/04/2025,Căn cứ chi bồi dưỡng cho diễn viên các hội thi...,Trung tâm chúng tôi chi bồi dưỡng cho diễn viê...,"Bộ Văn hóa, Thể thao và Du lịch trả lời vấn đề..."
1,Nguyễn Chánh,10:05 11/04/2025,Thủ tục hải quan đối với hàng gia công cho thư...,"Tôi xin hỏi, công ty có vốn đầu tư nước ngoài ...",Bộ Tài chính trả lời vấn đề này như sau:Căn cứ...
2,Bùi Quang Huy,09:25 11/04/2025,Xác định loại gói thầu theo căn cứ nào?,"Tôi xin hỏi, gói thầu sửa chữa, cải tạo sơn nư...",Bộ Kế hoạch và Đầu tư (nay là Bộ Tài chính) tr...
3,Võ Hồng Liên,07:05 10/04/2025,Có được yêu cầu nhà thầu bổ sung kê khai nhân ...,"Tôi xin hỏi, nhà thầu không kê khai nhân sự ch...",Bộ Kế hoạch và Đầu tư (nay là Bộ Tài chính) có...
4,Nguyễn Xuân Bình,08:46 09/04/2025,Có được chỉ định gói thầu sử dụng vốn sản xuất...,"Công ty tôi 100% vốn nhà nước, tổ chức cho ngư...",Bộ Kế hoạch và Đầu tư (nay là Bộ Tài chính) tr...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../Source_Fine-Tuning/data/taichinh_nganhang_dautu_congthuong_data.csv'
df_taichinh_nganhang.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../Source_Fine-Tuning/data/taichinh_nganhang_dautu_congthuong_data.csv


In [6]:
df_taichinh_nganhang=pd.read_csv("../Source_Fine-Tuning/data/taichinh_nganhang_dautu_congthuong_data.csv",encoding='utf-8')

In [8]:
df_taichinh_nganhang.shape

(2999, 6)